<a href="https://colab.research.google.com/github/ENEE439d-TEXTML/TextML/blob/master/data2_citation_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [14]:
address = "/content/drive/MyDrive/Maryland/4 Spring/ENEE439D/Text-ML/Jie_Tang_1_Article_All_Part00000.csv"
oldnum = "00000"
#list of csv numbers in drive
numbers = ["40002","40009", "40010", "40011"]
frequency_list = []
cite_list = []

for number in numbers:
  address = address.replace(oldnum, number)
  oldnum = number
  #read in csv
  data2  = pd.read_csv(address, header=None)
  


  #extract columns
  column0 = data2[0] #title data
  column1 = data2[1] #section data
  sentencedata2 = data2[2] #actual sentences

# The following section is used to extract the number of papers that are referenced in an article
  y = 0
  for x in column0:
    if x == 'references': #find where the number of references are listed
      break
    y = y+1 #increment iterator
  references = column1[y] #references = column1[column0 == 'references']




  #create array that is length of number of references in order to keep track of how many times referenced
  frequency = np.zeros(int(references))


  notempty = sentencedata2.notna() #find nonempty sentences
  sentencedata2=sentencedata2[notempty]
  
  citations = []
  for sentence in sentencedata2:
    matches = re.findall("\[\d+\]",sentence) # find [#]
    if  matches: #if a reference is found
      #increment frequency array at given spot based on reference found
      for match in matches:
        match = int(match.strip("[]")) 
        frequency[match-1] = frequency[match-1] + 1

      #store sentence in list
      citations.append(sentence)

  print(citations)
  print(frequency)
  
  #store list in list
  cite_list.append(citations)
  frequency_list.append(frequency)
  

#cite_list
#print(frequency_list)

['One of the major factors is the thermal stress and strain induced in packaging process [1],[2].', 'In previous research works, some mainly focused on the packaging effect on the electromechanical characteristics of conventional MEMS devices such as pull-in voltage [3].', 'Others only pay attention to the influence of different packaging structures and packaging materials on the microwave performance of RF MEMS devices [4],[5].', 'It can be seen from Fig. 1 that beam-like elements constitute the major mechanical parts of the model, including the switch beam, chip substrate, adhesive layer, and package substrate [6].', 'In the E-M domain, the phase shifter can well be modeled as an equivalent circuit of a cascade combination of the two elements, as shown in Fig. 4 [7], [8].']
[1. 1. 1. 1. 1. 1. 1. 1. 0.]
['The first MEMS gyroscope was introduced by the Draper Laboratory in the early 90’s [1].', 'The performance of micro-gyroscopes is limited by long-term drift, where current state-of-t